# Cleaning the Data

## Data Prep 

### Load Packages

In [10]:
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import Image
from ipywidgets import interact, fixed

In [16]:
df = pd.read_csv("SampleData.csv")
df.head()

,ID Único,Data,Unidade,Grupo EFR,Grupo Rúbrica,Tipo Rúbrica,Sexo,Data de Nascimento,Unnamed: 8,Unnamed: 9
0,1,20-02-2017,HCIS,PARTICULARES,RX CONVENCIONAL,CONSULTA EXTERNA,F,1961,NaN,NaN
1,1,26-07-2017,CCA,PARTICULARES,URGÊNCIA GERAL,URGÊNCIAS,F,1961,NaN,This is just some randomly generated data. It ...
2,1,04-08-2017,HCS,ADSE,GASTROENTEROLOGIA,IMAGIOLOGIA,F,1961,NaN,NaN
3,1,15-09-2017,HCIS,ADSE,RECOBRO,NaN,F,1961,NaN,NaN
4,2,12-01-2017,HCS,ADSE,NEURO-CIRURGIA,URGÊNCIAS,F,1971,NaN,NaN


### Manipulate Columns

In [17]:
#Get rid of blank columns
df_col= df.columns[0:8]
df_col
df = df[df_col]
df.head()

,ID Único,Data,Unidade,Grupo EFR,Grupo Rúbrica,Tipo Rúbrica,Sexo,Data de Nascimento
0,1,20-02-2017,HCIS,PARTICULARES,RX CONVENCIONAL,CONSULTA EXTERNA,F,1961
1,1,26-07-2017,CCA,PARTICULARES,URGÊNCIA GERAL,URGÊNCIAS,F,1961
2,1,04-08-2017,HCS,ADSE,GASTROENTEROLOGIA,IMAGIOLOGIA,F,1961
3,1,15-09-2017,HCIS,ADSE,RECOBRO,NaN,F,1961
4,2,12-01-2017,HCS,ADSE,NEURO-CIRURGIA,URGÊNCIAS,F,1971


In [19]:
#Translate column names into english
data = df.rename(columns={
    "ID Único":"UniqueID",
    "Data":"Date", 
    "Unidade":"Facility",
    "Grupo EFR":"Payer",
    "Grupo Rúbrica":"SpecificService", 
    "Tipo Rúbrica":"CategoryofService",
    "Sexo":"Sex",
    "Data de Nascimento":"BirthYear",
})

data.head(4)

,UniqueID,Date,Facility,Payer,SpecificService,CategoryofService,Sex,BirthYear
0,1,20-02-2017,HCIS,PARTICULARES,RX CONVENCIONAL,CONSULTA EXTERNA,F,1961
1,1,26-07-2017,CCA,PARTICULARES,URGÊNCIA GERAL,URGÊNCIAS,F,1961
2,1,04-08-2017,HCS,ADSE,GASTROENTEROLOGIA,IMAGIOLOGIA,F,1961
3,1,15-09-2017,HCIS,ADSE,RECOBRO,NaN,F,1961


In [20]:
# Calculate Approximate Age
data["Age"]= 2019 - data["BirthYear"]

In [21]:
# Delete Uneeded Columns
del data["BirthYear"]
del data["Date"]

### Data Cleaning

In [ ]:
# Add when get real data

### Export Dataset

In [23]:
#Use pickle format so that df reads into notebook faster 
data.to_pickle("cleaned_data.pkl")